In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.insert(0, "/notebooks/")
from fastai.imports import *

In [5]:
from fastai.core import *
from fastai.transforms import *


model_meta = {
	resnet18:[8,6], resnet34:[8,6], resnet50:[8,6], resnet101:[8,6], resnet152:[8,6],
	vgg16:[0,22], vgg19:[0,22],
	resnext50: [8, 6], resnext101: [8, 6], resnext101_64: [8, 6],
	dn121:[0,7], dn161:[0,7], dn169:[0,7], dn201:[0,7],
}

model_features = {}  # {inception_4: 3072, dn121: 2048, dn161: 4416, nasnetalarge: 4032*2}


class ConvnetBuilder(object):
	"""Class representing a convolutional network.

	Arguments:
			f: a model creation function (e.g. resnet34, vgg16, etc)
			c (int): size of the last layer
			is_multi (bool): is multilabel classification?
					(def here http://scikit-learn.org/stable/modules/multiclass.html)
			is_reg (bool): is a regression?
			ps (float or array of float): dropout parameters
			xtra_fc (list of ints): list of hidden layers with # hidden neurons
			xtra_cut (int): # layers earlier than default to cut the model, default is 0
			custom_head : add custom model classes that are inherited from nn.modules at the end of the model
										that is mentioned on Argument 'f'
	"""

	def __init__(self, f, c, is_multi, is_reg, ps=None, xtra_fc=None, xtra_cut=0, custom_head=None, pretrained=True):
		self.f, self.c, self.is_multi, self.is_reg, self.xtra_cut = f, c, is_multi, is_reg, xtra_cut
		if xtra_fc is None: xtra_fc = [512]
		if ps is None: ps = [0.25]*len(xtra_fc)+[0.5]
		self.ps, self.xtra_fc = ps, xtra_fc

		if f in model_meta:
			cut, self.lr_cut = model_meta[f]
		else:
			cut, self.lr_cut = 0, 0
		cut -= xtra_cut
		layers = cut_model(f(pretrained), cut)
		self.nf = model_features[f] if f in model_features else (num_features(layers)*2)
		if not custom_head: layers += [AdaptiveConcatPool2d(), Flatten()]
		self.top_model = nn.Sequential(*layers)

		n_fc = len(self.xtra_fc)+1
		if not isinstance(self.ps, list): self.ps = [self.ps]*n_fc

		if custom_head:
			fc_layers = [custom_head]
		else:
			fc_layers = self.get_fc_layers()
		self.n_fc = len(fc_layers)
		self.fc_model = to_gpu(nn.Sequential(*fc_layers))
		if not custom_head: apply_init(self.fc_model, kaiming_normal)
		self.model = to_gpu(nn.Sequential(*(layers+fc_layers)))

	@property
	def name(self):
		return f'{self.f.__name__}_{self.xtra_cut}'

	def create_fc_layer(self, ni, nf, p, actn=None):
		res = [nn.BatchNorm1d(num_features=ni)]
		if p: res.append(nn.Dropout(p=p))
		res.append(nn.Linear(in_features=ni, out_features=nf))
		if actn: res.append(actn)
		return res

	def get_fc_layers(self):
		res = []
		ni = self.nf
		for i, nf in enumerate(self.xtra_fc):
			res += self.create_fc_layer(ni, nf, p=self.ps[i], actn=nn.ReLU())
			ni = nf
		final_actn = nn.Sigmoid() if self.is_multi else nn.LogSoftmax()
		if self.is_reg: final_actn = None
		res += self.create_fc_layer(ni, self.c, p=self.ps[-1], actn=final_actn)
		return res

	def get_layer_groups(self, do_fc=False):
		if do_fc:
			return [self.fc_model]
		idxs = [self.lr_cut]
		c = children(self.top_model)
		if len(c) == 3: c = children(c[0])+c[1:]
		lgs = list(split_by_idxs(c, idxs))
		return lgs+[self.fc_model]


In [6]:
head_reg4 = nn.Sequential(
    Flatten(),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(25088,256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(0.5),
    nn.Linear(256,4+20),
)

In [8]:
def classification_model(arch=resnet34, **kwargs):
	opts = dict(is_multi=False, is_reg=False, pretrained=False)
	conv = ConvnetBuilder(arch, 0, **opts, **kwargs)
	return conv.model

In [11]:
import os
import boto3
import cv2
import numpy as np
import urllib.request


s3_client = boto3.client('s3')


def download_file(bucket_name, object_key_name, file_path):
	"""  Downloads a file from an S3 bucket.
	:param bucket_name: S3 bucket name
	:param object_key_name: S3 object key name
	:param file_path: path to save downloaded file
	"""
	s3_client.download_file(bucket_name, object_key_name, file_path)


def get_labels(path):
	"""  Get labels from a text file.
	:param path: path to text file
	:return: list of labels
	"""
	with open(path, encoding='utf-8', errors='ignore') as f:
		labels = [line.strip() for line in f.readlines()]
		f.close()
	return labels


def open_image_url(url):
	"""  Opens an image using OpenCV from a URL.
	:param url: url path of the image
	:return: the image in RGB format as numpy array of floats normalized to range between 0.0 - 1.0
	"""
	flags = cv2.IMREAD_UNCHANGED+cv2.IMREAD_ANYDEPTH+cv2.IMREAD_ANYCOLOR
	url = str(url)
	resp = urllib.request.urlopen(url)
	try:
		im = np.asarray(bytearray(resp.read()))
		im = cv2.imdecode(im, flags).astype(np.float32)/255
		if im is None: raise OSError(f'File from url not recognized by opencv: {url}')
		return im
	except Exception as e:
		raise OSError(f'Error handling image from url at: {url}') from e


def open_image(path):
	""" Opens an image using OpenCV given the file path.
	:param path: the file path of the image
	:return: the image in RGB format as numpy array of floats normalized to range between 0.0 - 1.0
	"""
	flags = cv2.IMREAD_UNCHANGED+cv2.IMREAD_ANYDEPTH+cv2.IMREAD_ANYCOLOR
	path = str(path)
	if not os.path.exists(path):
		raise OSError(f'No such file or directory: {path}')
	elif os.path.isdir(path):
		raise OSError(f'Is a directory: {path}')
	else:
		try:
			im = cv2.imread(str(path), flags).astype(np.float32)/255
			if im is None: raise OSError(f'File not recognized by opencv: {path}')
			return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
		except Exception as e:
			raise OSError(f'Error handling image at: {path}') from e


In [17]:
BUCKET_NAME = 'pytorch-serverless-shun'
STATE_DICT_NAME = 'pascal_1.h5'
SZ = 224
IMAGE_STATS = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
LABELS_PATH = 'lib/labels.txt'

In [21]:
import os, warnings
import torch, torchvision

from torch.autograd import Variable
from torch.nn.init import kaiming_normal

from torchvision.transforms import Compose
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152
from torchvision.models import vgg16_bn, vgg19_bn
from torchvision.models import densenet121, densenet161, densenet169, densenet201

from .models.resnext_50_32x4d import resnext_50_32x4d
from .models.resnext_101_32x4d import resnext_101_32x4d
from .models.resnext_101_64x4d import resnext_101_64x4d


warnings.filterwarnings('ignore', message='Implicit dimension choice', category=UserWarning)


def children(m): return m if isinstance(m, (list, tuple)) else list(m.children())


def save_model(m, p): torch.save(m.state_dict(), p)


def load_model(m, p): m.load_state_dict(torch.load(p, map_location=lambda storage, loc: storage))


def load_pre(pre, f, fn):
	m = f()
	path = os.path.dirname(__file__)
	if pre: load_model(m, f'{path}/weights/{fn}.pth')
	return m


def _fastai_model(name, paper_title, paper_href):
	def add_docs_wrapper(f):
		f.__doc__ = f"""{name} model from
        `"{paper_title}" <{paper_href}>`_

        Args:
           pre (bool): If True, returns a model pre-trained on ImageNet
        """
		return f

	return add_docs_wrapper


@_fastai_model('ResNeXt 50', 'Aggregated Residual Transformations for Deep Neural Networks',
               'https://arxiv.org/abs/1611.05431')
def resnext50(pre): return load_pre(pre, resnext_50_32x4d, 'resnext_50_32x4d')


@_fastai_model('ResNeXt 101_32', 'Aggregated Residual Transformations for Deep Neural Networks',
               'https://arxiv.org/abs/1611.05431')
def resnext101(pre): return load_pre(pre, resnext_101_32x4d, 'resnext_101_32x4d')


@_fastai_model('ResNeXt 101_64', 'Aggregated Residual Transformations for Deep Neural Networks',
               'https://arxiv.org/abs/1611.05431')
def resnext101_64(pre): return load_pre(pre, resnext_101_64x4d, 'resnext_101_64x4d')


@_fastai_model('Densenet-121', 'Densely Connected Convolutional Networks',
               'https://arxiv.org/pdf/1608.06993.pdf')
def dn121(pre): return children(densenet121(pre))[0]


@_fastai_model('Densenet-169', 'Densely Connected Convolutional Networks',
               'https://arxiv.org/pdf/1608.06993.pdf')
def dn161(pre): return children(densenet161(pre))[0]


@_fastai_model('Densenet-161', 'Densely Connected Convolutional Networks',
               'https://arxiv.org/pdf/1608.06993.pdf')
def dn169(pre): return children(densenet169(pre))[0]


@_fastai_model('Densenet-201', 'Densely Connected Convolutional Networks',
               'https://arxiv.org/pdf/1608.06993.pdf')
def dn201(pre): return children(densenet201(pre))[0]


@_fastai_model('Vgg-16 with batch norm added', 'Very Deep Convolutional Networks for Large-Scale Image Recognition',
               'https://arxiv.org/pdf/1409.1556.pdf')
def vgg16(pre): return children(vgg16_bn(pre))[0]


@_fastai_model('Vgg-19 with batch norm added', 'Very Deep Convolutional Networks for Large-Scale Image Recognition',
               'https://arxiv.org/pdf/1409.1556.pdf')
def vgg19(pre): return children(vgg19_bn(pre))[0]


ModuleNotFoundError: No module named '__main__.models'; '__main__' is not a package

In [19]:
from .torch_imports import children


def cut_model(m, cut):
	return list(m.children())[:cut] if cut else [m]


def num_features(m):
	c = children(m)
	if len(c) == 0: return None
	for l in reversed(c):
		if hasattr(l, 'num_features'): return l.num_features
		res = num_features(l)
		if res is not None: return res


ModuleNotFoundError: No module named '__main__.torch_imports'; '__main__' is not a package

In [20]:
import os, json, traceback
import urllib.parse
import torch
import numpy as np
from scipy.special import expit

from fastai.core import A, T, VV_
from fastai.transforms import tfms_from_stats


STATS = A(IMAGE_STATS)
TFMS = tfms_from_stats(STATS, SZ)[-1]


class SetupModel(object):
	model = classification_model()
	labels = get_labels(os.environ['LABELS_PATH'])

	def __init__(self, f):
		self.f = f
		file_path = f'/tmp/{STATE_DICT_NAME}'
		download_file(BUCKET_NAME, STATE_DICT_NAME, file_path)
		state_dict = torch.load(file_path, map_location=lambda storage, loc: storage)
		self.model.load_state_dict(state_dict), self.model.eval()
		os.remove(file_path)

	def __call__(self, *args, **kwargs):
		return self.f(*args, **kwargs)


def build_pred(label_idx, bb):
	label = SetupModel.labels[label_idx]
	return dict(label=label, bb=bb)


def parse_params(params):
	image_url = urllib.parse.unquote_plus(params.get('image_url', ''))
	return dict(image_url=image_url)


def predict(img):
	batch = [T(TFMS(img))]
	inp = VV_(torch.stack(batch))
	return SetupModel.model(inp).mean(0)

def bb_to_list(bb):
	bb_list=[]
	for b in bb:
		bb_list.append(b.item())
	return bb_list


@SetupModel
def handler(event, _):
	if event is None: event = {}
	print(event)
	try:
		# keep the lambda function warm
		if event.get('detail-type') is 'Scheduled Event':
			return 'nice & warm'

		params = parse_params(event.get('queryStringParameters', {}))
		out = predict(open_image_url(params['image_url']))

		out = out.data.numpy
		bb = expit(out[:4])*224
		bb = bb_to_list(bb)
		c = out[4:]
		c = np.argmax(c)

		preds = [build_pred(c, bb)]

		response_body = dict(predictions=preds)
		response = dict(statusCode=200, body=response_body)

	except Exception as e:
		response_body = dict(error=str(e), traceback=traceback.format_exc())
		response = dict(statusCode=500, body=response_body)

	response['body'] = json.dumps(response['body'])
	print(response)
	return response

NameError: name 'cut_model' is not defined